# EOF + PC demo (full version)

All plotting functions + synthetic data.
Binder / Cartopy ready.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import cmocean
from matplotlib.gridspec import GridSpec

plt.rcParams.update({
    'font.family': 'sans-serif',
    'font.sans-serif': ['DejaVu Sans'],
    'font.size': 8,
    'axes.linewidth': 0.5,
    'xtick.major.width': 0.5,
    'ytick.major.width': 0.5,
    'figure.dpi': 300,
    'savefig.dpi': 300,
    'savefig.bbox': 'tight',
    'savefig.pad_inches': 0.05
})

In [ ]:
def plot_eof_spatial_pattern(eof, lons, lats, mode_num, variance_explained,
                             save_path='eof_spatial.pdf'):

    fig = plt.figure(figsize=(7, 4))
    proj = ccrs.PlateCarree(central_longitude=180)
    ax = plt.axes(projection=proj)

    ax.set_extent([120, 290, -60, 60], crs=ccrs.PlateCarree())
    levels = np.linspace(-np.abs(eof).max(), np.abs(eof).max(), 21)

    cf = ax.contourf(lons, lats, eof, levels=levels,
                     cmap=cmocean.cm.balance,
                     transform=ccrs.PlateCarree(), extend='both')

    ax.contour(lons, lats, eof, levels=levels[::2], colors='k',
               linewidths=0.3, alpha=0.3,
               transform=ccrs.PlateCarree())

    ax.coastlines(linewidth=0.5)
    ax.add_feature(cfeature.LAND, facecolor='lightgray', zorder=0)

    gl = ax.gridlines(draw_labels=True, linewidth=0.3,
                      color='gray', alpha=0.5, linestyle='--')
    gl.top_labels = gl.right_labels = False
    gl.xformatter = LongitudeFormatter()
    gl.yformatter = LatitudeFormatter()

    cbar = plt.colorbar(cf, ax=ax, orientation='horizontal', pad=0.05)
    cbar.set_label('SST anomaly (°C)')

    ax.set_title(f'EOF {mode_num} ({variance_explained:.1f}%)',
                 fontweight='bold')

    plt.savefig(save_path)
    plt.close()

In [ ]:
def plot_pc_timeseries(pc, time, mode_num, save_path='pc_timeseries.pdf'):

    fig, ax = plt.subplots(figsize=(7, 2.5))
    pc_norm = (pc - pc.mean()) / pc.std()

    ax.fill_between(time, 0, pc_norm, where=(pc_norm >= 0),
                    color='#d73027', alpha=0.7)
    ax.fill_between(time, 0, pc_norm, where=(pc_norm < 0),
                    color='#4575b4', alpha=0.7)

    ax.plot(time, pc_norm, 'k-', linewidth=0.5)
    ax.axhline(0, color='k', linewidth=0.5, alpha=0.3)

    ax.set_xlabel('Year')
    ax.set_ylabel(f'PC{mode_num} (norm)')
    ax.grid(True, alpha=0.2)

    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()

In [ ]:
def plot_eof_summary(eof_modes, pcs, variance_explained, lons, lats, time,
                     n_modes=3, save_path='eof_summary.pdf'):

    fig = plt.figure(figsize=(7.2, 9))
    gs = GridSpec(n_modes, 2, figure=fig,
                  width_ratios=[1.5, 1], hspace=0.3, wspace=0.3)

    proj = ccrs.PlateCarree(central_longitude=180)

    for i in range(n_modes):
        ax_map = fig.add_subplot(gs[i, 0], projection=proj)
        eof = eof_modes[i]
        levels = np.linspace(-np.abs(eof).max(), np.abs(eof).max(), 21)

        cf = ax_map.contourf(lons, lats, eof, levels=levels,
                             cmap=cmocean.cm.balance,
                             transform=ccrs.PlateCarree(), extend='both')

        ax_map.set_extent([120, 290, -60, 60], crs=ccrs.PlateCarree())
        ax_map.coastlines(linewidth=0.4)
        ax_map.add_feature(cfeature.LAND, facecolor='lightgray')

        letter = chr(97 + i*2)
        ax_map.text(-0.05, 1.05, f'({letter})', transform=ax_map.transAxes,
                    fontsize=10, fontweight='bold')

        ax_map.set_title(f'EOF {i+1} ({variance_explained[i]:.1f}%)', fontsize=8)

        ax_ts = fig.add_subplot(gs[i, 1])
        pc_norm = (pcs[i] - pcs[i].mean()) / pcs[i].std()

        ax_ts.fill_between(time, 0, pc_norm, where=(pc_norm >= 0),
                           color='#d73027', alpha=0.7)
        ax_ts.fill_between(time, 0, pc_norm, where=(pc_norm < 0),
                           color='#4575b4', alpha=0.7)

        ax_ts.plot(time, pc_norm, 'k-', linewidth=0.4)
        ax_ts.axhline(0, color='k', linewidth=0.4, alpha=0.3)

        letter = chr(97 + i*2 + 1)
        ax_ts.text(-0.15, 1.05, f'({letter})', transform=ax_ts.transAxes,
                   fontsize=10, fontweight='bold')

        if i == n_modes - 1:
            ax_ts.set_xlabel('Year')

    plt.savefig(save_path)
    plt.close()

In [ ]:
lats = np.linspace(-60, 60, 61)
lons = np.linspace(120, 290, 86)
lon2d, lat2d = np.meshgrid(lons, lats)

eof1 = np.cos(np.deg2rad(lat2d)) * np.sin(np.deg2rad(lon2d - 180))
eof2 = np.sin(2*np.deg2rad(lat2d)) * np.cos(np.deg2rad(lon2d - 200))
eof3 = np.cos(np.deg2rad(lat2d/2)) * np.sin(2*np.deg2rad(lon2d - 160))

eof_modes = [eof1, eof2, eof3]

time = np.arange(1980, 2021)
pcs = [
    np.sin(2*np.pi*(time-1980)/6)  + 0.3*np.random.randn(len(time)),
    np.sin(2*np.pi*(time-1980)/12) + 0.3*np.random.randn(len(time)),
    np.sin(2*np.pi*(time-1980)/20) + 0.3*np.random.randn(len(time))
]

variance_explained = np.array([35.0, 18.0, 10.0])

In [ ]:
plot_eof_summary(eof_modes, pcs, variance_explained,
                 lons, lats, time,
                 n_modes=3,
                 save_path='eof_summary.pdf')

print('All figures generated')